In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pw
import pandas as pd
from requests import exceptions

In [2]:

client_id = pw.client_id
client_secret = pw.client_secret
redirect_uri = pw.redirect_uri
user_id = pw.user_id
username = pw.username

In [3]:
token = util.prompt_for_user_token(user_id,
                                   'playlist-read-collaborative',
                                   client_id=client_id,
                                   client_secret=client_secret,
                                   redirect_uri=redirect_uri)
sp = spotipy.Spotify(auth=token)

In [5]:
def getPlaylists(userName):
    
    playlists = sp.user_playlists(userName)
    playlist_lst = []

    for playlist in playlists['items']:
        playlist_dict = {}
        playlist_dict['playlistName'] = playlist['name']
        playlist_dict['playlistId'] = playlist['id']
        playlist_dict['playlistHref'] = playlist['href']
        playlist_dict['playlisOwner'] = playlist['owner']['display_name']
        playlist_dict['playlisPublic'] = playlist['public']
        playlist_dict['tracksHref'] = playlist['tracks']['href']
        playlist_dict['tracksCount'] = playlist['tracks']['total']

        playlist_lst.append(playlist_dict)
    
    return pd.DataFrame.from_dict(playlist_lst)

getPlaylists('kocza.milan')

,playlisOwner,playlisPublic,playlistHref,playlistId,playlistName,tracksCount,tracksHref
0,Milán Kócza,True,https://api.spotify.com/v1/playlists/309rQiuXl...,309rQiuXlHQ1WqwkHouoJO,fuk.society,3,https://api.spotify.com/v1/playlists/309rQiuXl...
1,Milán Kócza,True,https://api.spotify.com/v1/playlists/4tEeo0kzK...,4tEeo0kzKunhBwsPYndOBz,Részegen hazafelé,41,https://api.spotify.com/v1/playlists/4tEeo0kzK...
2,Milán Kócza,True,https://api.spotify.com/v1/playlists/7bLyD3yD4...,7bLyD3yD4hSG0u3DeBCOKA,lépeget,4,https://api.spotify.com/v1/playlists/7bLyD3yD4...
3,Milán Kócza,True,https://api.spotify.com/v1/playlists/79huJWUFr...,79huJWUFredOrxzcXdCGYo,cool hits,187,https://api.spotify.com/v1/playlists/79huJWUFr...
4,Milán Kócza,True,https://api.spotify.com/v1/playlists/6eDPUoJUs...,6eDPUoJUsnA0580PYCXdTX,lépegetős,17,https://api.spotify.com/v1/playlists/6eDPUoJUs...
5,Bence Vitárius,True,https://api.spotify.com/v1/playlists/0xlgkjKX7...,0xlgkjKX7eUFYxeVRVWA7t,British Mood Top 250,278,https://api.spotify.com/v1/playlists/0xlgkjKX7...
6,Spotify,True,https://api.spotify.com/v1/playlists/37i9dQZF1...,37i9dQZF1DWSOkubnsDCSS,Grime Shutdown,81,https://api.spotify.com/v1/playlists/37i9dQZF1...


In [19]:
def getTracks(playlistId):
    
    sp = spotipy.Spotify(auth=token)
    track_lst = []
    sourcePlaylist = sp.user_playlist(user=username, playlist_id=playlistId)
    tracks = sourcePlaylist['tracks']
    
    while True:
        sp = spotipy.Spotify(auth=token)
        for track in tracks['items']:
            
            track_dict = {}

            artistId = track['track']['album']['artists'][0]['id']
            artistInfo = sp.artist(artistId)

            songId = track['track']['id']
            audioInfo = sp.audio_features(songId)[0]

            track_dict['playListId'] = playlistId
            #track_dict['playListName'] = playlist["playlistName"]
            track_dict['added_at'] = track['added_at']
            track_dict['artisId'] = artistId
            track_dict['artisName'] = track['track']['album']['artists'][0]['name']
            track_dict['artisType'] = track['track']['album']['artists'][0]['type']
            track_dict['genre'] = artistInfo['genres']
            track_dict['artistPopularity'] = artistInfo['popularity']
            track_dict['albumId'] = track['track']['album']['id']
            track_dict['albumName'] = track['track']['album']['name']
            track_dict['albumReleaseDate'] = track['track']['album']['release_date']
            track_dict['songId'] = songId
            track_dict['songTitle'] = track['track']['name']
            track_dict['songPopularity'] = track['track']['popularity']
            track_dict['songDurationMs'] = track['track']['duration_ms']
            track_dict['danceability'] = audioInfo['danceability']
            track_dict['energy'] = audioInfo['energy']
            track_dict['loudness'] = audioInfo['loudness']
            track_dict['mode'] = audioInfo['mode']
            track_dict['speechiness'] = audioInfo['speechiness']
            track_dict['acousticness'] = audioInfo['acousticness']
            track_dict['instrumentalness'] = audioInfo['instrumentalness']
            track_dict['liveness'] = audioInfo['liveness']
            track_dict['valence'] = audioInfo['valence']
            track_dict['tempo'] = audioInfo['tempo'] 

            track_lst.append(track_dict)

        if tracks['next'] == None:
            break

        tracks = sp._get(tracks['next'])
    
    return pd.DataFrame.from_dict(track_lst)

df = getTracks('79huJWUFredOrxzcXdCGYo')


retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs


In [23]:
data = df[['albumName', 'artisName', 'songTitle', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo',  'valence']]
data

,albumName,artisName,songTitle,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,valence
0,Feel It Still,Portugal. The Man,Feel It Still,0.040600,0.780,0.798,0.000063,0.0733,-5.151,0,0.0613,78.969,0.7710
1,Sirens,Nicolas Jaar,No,0.802000,0.583,0.409,0.240000,0.1390,-15.926,0,0.3920,89.887,0.1860
2,Flume (Deluxe Edition),Flume,Bring You Down,0.236000,0.436,0.627,0.120000,0.3640,-6.703,0,0.0274,169.998,0.1350
3,The North Borders,Bonobo,Cirrus,0.395000,0.640,0.844,0.933000,0.0827,-8.412,0,0.0374,119.042,0.3640
4,Woman,Rhye,The Fall,0.342000,0.652,0.621,0.651000,0.1770,-12.733,1,0.0380,104.996,0.7500
5,I'm Leaving,IS TROPICAL,Dancing Anymore,0.000425,0.561,0.724,0.493000,0.1210,-6.590,1,0.0274,127.014,0.3860
6,Tohu Bohu,Rone,Bye Bye Macadam,0.107000,0.593,0.509,0.893000,0.2770,-12.042,0,0.0626,190.028,0.0769
7,Tape Two,Young Fathers,Queen Is Dead,0.003610,0.367,0.794,0.000158,0.3790,-3.273,1,0.2530,117.388,0.2980
8,White Men Are Black Men Too,Young Fathers,Dare Me,0.143000,0.521,0.671,0.002160,0.3490,-3.654,1,0.0924,85.819,0.0734
9,White Men Are Black Men Too,Young Fathers,Nest,0.046300,0.504,0.885,0.000000,0.1800,-4.625,0,0.0420,115.973,0.3280
